In [5]:
def FullPPR(train_features, train_labels, test_features, test_labels, train_pixels, rtry, runname,
            PredsNames, degtry = [3], maxperpage = 5, resplabel='Log Size'):
    
    from sklearn.model_selection import KFold, GroupKFold
    from sklearn.model_selection import cross_val_score
    from skpp import ProjectionPursuitRegressor

    
    #----------------------------------------------------------------------------------------
    
    print('-----------------------------------------------')
    print('Histograms of Distributions of Each Predictor')
    
    #for pred in train_features.columns:
    #    plt.figure(figsize=[5,5])
    #     hist(subinfo[pred], bins=100)
    #    plt.xlabel(pred,size=12)
    #    plt.show()

        
    #----------------------------------------------------------------------------------------
    
    if(len(rtry)*len(degtry) > 1):
        print('-----------------------------------------------')
        print('Running Cross-Validation Procedure')
    
        minScore = Inf

        resmat = np.empty([len(degtry)*len(rtry), 4])

        pos = 0

        for rcand in rtry:
            for degcand in degtry:
                # Assign data to folds
                #define cross-validation method to use
                #cv = KFold(n_splits=10, random_state=1, shuffle=True)
                cv = GroupKFold(n_splits=10)

                model = ProjectionPursuitRegressor(r=rcand, show_plots=False,
                    degree=degcand, fit_type='spline',
                    random_state=globalseed)

                #use k-fold CV to evaluate model
                scores = absolute(cross_val_score(model, train_features, train_labels,
                         cv=cv, n_jobs=-1, scoring='neg_mean_squared_error',
                         groups=train_pixels))

                #store results
                resmat[pos,0] = rcand
                resmat[pos,1] = degcand
                resmat[pos,2] = mean(scores)
                resmat[pos,3] = resmat[pos,2] + sqrt(var(scores)/10)
        
                pos += 1
        
                print(resmat[:pos])


        # Using the One-SE rule. 
        # See Slide 20 of https://www.stat.cmu.edu/~ryantibs/datamining/lectures/18-val1.pdf

        ruse = int(resmat[resmat[:,2] <= resmat[min(resmat[:,2]) == resmat[:,2],3]][0][0])
        deguse = int(resmat[resmat[:,2] <= resmat[min(resmat[:,2]) == resmat[:,2],3]][0][1])
 
    else:
        ruse = rtry[0]
        deguse = degtry[0]
    

    #----------------------------------------------------------------------------------------
    
    print('-----------------------------------------------')

    print('Fitting the model with r =',ruse,'and degree =', deguse)

    PPRfullfit = ProjectionPursuitRegressor(r=ruse,
        show_plots=False, degree=deguse, fit_type='spline',
        random_state=globalseed)

    PPRfullfit.fit(train_features,train_labels)
    
    # This is the projection of the data using the alpha vectors, i.e., this is prior to
    # the application of the nonlinear functions.
    
    projs = PPRfullfit.transform(train_features)
    
    
    # Determine the fits from each projection.
    
    fitsbyproj = projs.copy()
    for i in range(ruse):
        fitsbyproj[:,i] = PPRfullfit._f[i](projs[:,i])*PPRfullfit._beta[0][i]
       
    
    # Calculate the fitted values and residuals. Note that fitted values could also be
    # found from PPRfullfit.predict(train_features).
    
    fittedvalues = fitsbyproj.sum(1)
    residuals = train_labels - fittedvalues
    
    
    #----------------------------------------------------------------------------------------
    
    print('-----------------------------------------------')
    print('Creating the Plots showing the Ridge Functions.')

    fignum = 0

    for i in range(ruse):
    
        pos = i % maxperpage

        if(pos == 0):
            if(i > 0):
                plt.savefig(runname+'Fit_plot'+str(int(floor((i-1)/maxperpage)))+'.png')
            fignum += 1
            fig,axs = plt.subplots(min(ruse-i, maxperpage),2,figsize=[12,3.8*min(ruse-i,maxperpage)])
            if(ruse == (i+1)):
                axs = np.expand_dims(axs,0)
        
        # The residuals from the fit leaving out projection i
    
        residswithouti = residuals + fitsbyproj[:,i]
    
        axs[pos,0].barh(PredsNames, PPRfullfit._alpha[:,i])
        axs[pos,0].plot([0, 0], [-5, len(PredsNames)+5], 'k-')
        axs[pos,0].set(ylim=(-1,len(PredsNames)))
    
        axs[pos,1].hist2d(x=projs[:,i],
           y=residswithouti,
             bins=300,
             norm=matplotlib.colors.LogNorm())

        foo = PPRfullfit._f[i](sort(projs[:,i]))*PPRfullfit._beta[0][i]

        axs[pos,1].plot(sort(projs[:,i]),foo,color="r")
        plt.title('')
        plt.ylabel('Residuals',size=12)
        if(ruse == 1):
            axs[pos,1].set(xlabel=('Projection '+str(i+1)), ylabel='Log Size')
        else:
            axs[pos,1].set(xlabel=('Projection '+str(i+1)), ylabel='Residuals')
    

    plt.savefig(runname+'Fit_plot'+str(int(floor(i/maxperpage)))+'.png')
    
    
    #----------------------------------------------------------------------------------------

    print('-----------------------------------------------')

    print('Plot of residuals versus fitted values')

    plt.figure(figsize=[5,5])
    hist2d(x=fittedvalues,
       y=residuals,
         bins=200,
         norm=matplotlib.colors.LogNorm())
    plt.title('Projection Pursuit Residuals versus Fitted Values')
    plt.ylabel('Residuals',size=12)
    plt.xlabel('Fitted Values',size=12)

    plt.show()
    
    print('Plot of actual response versus fitted values')
    
    plt.figure(figsize=[5,5])
    hist2d(x=fittedvalues,
       y=train_labels,
         bins=200,
         norm=matplotlib.colors.LogNorm())
    plt.title('Projection Pursuit Performance on Training Set')
    plt.ylabel(resplabel, size=12)
    plt.xlabel('PPR Fitted Value',size=12)
    plt.plot([-100, 100], [-100, 100], 'b-')

    plt.show()
    
    
    #----------------------------------------------------------------------------------------

    print('-----------------------------------------------')

    print('Performance on Test Set')

    PPRfitsontest = PPRfullfit.predict(test_features)
    
    plt.figure(figsize=[5,5])
    hist2d(x=PPRfitsontest,
       y=test_labels,
         bins=200,
         norm=matplotlib.colors.LogNorm())
    plt.title('Projection Pursuit Performance on Test Set')
    plt.ylabel('Log Size',size=12)
    plt.xlabel('PPR Prediction',size=12)
    plt.plot([-100, 100], [-100, 100], 'b-')
    plt.xlim(-1.5,3)
    plt.ylim(-1.5,3)

    plt.show()
    
    
    print("RMSE on test set:",round(sqrt(mean((PPRfitsontest-test_labels)**2)),3))

    print("RMSE on training set:",round(sqrt(mean(residuals**2)),3))
    
    return PPRfullfit